# 모듈 임포트

In [25]:
import pandas as pd
import numpy as np
import transformers
import os 
import re
from tqdm import tqdm
from transformers import pipeline

In [26]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("nbroad/ESG-BERT")

model = AutoModelForSequenceClassification.from_pretrained("nbroad/ESG-BERT")

# 뉴스 데이터 로드

In [27]:
nytimes_df = pd.read_csv('/Users/parkjunhyeong/Desktop/박준형/02. 대내 및 대외활동/01. 대내활동/03. Biz&AI 랩/02. 소스/01. 데이터/06. 회귀분석 데이터/Composed_Data_ver1.4.csv', index_col = 0)

In [28]:
nytimes_df.dropna(inplace = True)

In [29]:
nytimes_df.reset_index(drop = True, inplace = True)

In [11]:
nytimes_df['Main Text'].iloc[0]

'Netflix has blocked an episode of its show _atriot Act With Hasan Minhaj from streaming in Saudi Arabia after the Saudi government complained that the episode  which is critical of the crown prince, Mohammed bin Salman  violated its cybercrime laws.\n \n In the episode, first shown in October, Mr. Minhaj critiques the United States longstanding relationship with Saudi Arabia after the murder of the dissident Saudi journalist Jamal Khashoggi.\n \n _ow would be a good time to reassess our relationship with Saudi Arabia, Mr. Minhaj said, _nd I mean that as a Muslim and an American.\n \n After receiving a takedown request last month from the Saudi government_ Communications and Information Technology Commission, Netflix removed the episode from viewing in Saudi Arabia last week. The news was first reported by The Financial Times.'

# 데이터 전처리 및 자르기

In [12]:
def clean_text(sent):
    sent_clean = re.sub('\n\n', ' ', sent)
    sent_clean = re.sub('[^a-zA-Z]', ' ', sent)
    return sent_clean

In [13]:
nytimes_df['clean_text'] = nytimes_df['Main Text'].apply(lambda x : clean_text(str(x)))

In [14]:
nytimes_df['clean_text'].iloc[0]

'Netflix has blocked an episode of its show  atriot Act With Hasan Minhaj from streaming in Saudi Arabia after the Saudi government complained that the episode  which is critical of the crown prince  Mohammed bin Salman  violated its cybercrime laws     In the episode  first shown in October  Mr  Minhaj critiques the United States longstanding relationship with Saudi Arabia after the murder of the dissident Saudi journalist Jamal Khashoggi      ow would be a good time to reassess our relationship with Saudi Arabia  Mr  Minhaj said   nd I mean that as a Muslim and an American     After receiving a takedown request last month from the Saudi government  Communications and Information Technology Commission  Netflix removed the episode from viewing in Saudi Arabia last week  The news was first reported by The Financial Times '

In [15]:
nytimes_df['clean_text'] = nytimes_df['clean_text'].str.split(' ')

In [16]:
remove_set = ['']
for i in range(len(nytimes_df)):
    nytimes_df['clean_text'][i] = [l for l in nytimes_df['clean_text'][i] if l not in remove_set]

/var/folders/z6/nn0fl27x39547ybzc5y11s8m0000gn/T/ipykernel_12742/662184775.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nytimes_df['clean_text'][i] = [l for l in nytimes_df['clean_text'][i] if l not in remove_set]


In [17]:
nytimes_df['clean_text'][0]

['Netflix',
 'has',
 'blocked',
 'an',
 'episode',
 'of',
 'its',
 'show',
 'atriot',
 'Act',
 'With',
 'Hasan',
 'Minhaj',
 'from',
 'streaming',
 'in',
 'Saudi',
 'Arabia',
 'after',
 'the',
 'Saudi',
 'government',
 'complained',
 'that',
 'the',
 'episode',
 'which',
 'is',
 'critical',
 'of',
 'the',
 'crown',
 'prince',
 'Mohammed',
 'bin',
 'Salman',
 'violated',
 'its',
 'cybercrime',
 'laws',
 'In',
 'the',
 'episode',
 'first',
 'shown',
 'in',
 'October',
 'Mr',
 'Minhaj',
 'critiques',
 'the',
 'United',
 'States',
 'longstanding',
 'relationship',
 'with',
 'Saudi',
 'Arabia',
 'after',
 'the',
 'murder',
 'of',
 'the',
 'dissident',
 'Saudi',
 'journalist',
 'Jamal',
 'Khashoggi',
 'ow',
 'would',
 'be',
 'a',
 'good',
 'time',
 'to',
 'reassess',
 'our',
 'relationship',
 'with',
 'Saudi',
 'Arabia',
 'Mr',
 'Minhaj',
 'said',
 'nd',
 'I',
 'mean',
 'that',
 'as',
 'a',
 'Muslim',
 'and',
 'an',
 'American',
 'After',
 'receiving',
 'a',
 'takedown',
 'request',
 'last',

In [18]:
nytimes_df['truncated_text'] = nytimes_df['clean_text'].apply(lambda x : x[:400])

In [19]:
nytimes_df['clean_text'] = nytimes_df['clean_text'].str.join(' ')
nytimes_df['truncated_text'] = nytimes_df['truncated_text'].str.join(' ')

In [20]:
nytimes_df

,URL,Publish Date,Title,Main Text,Ticker,Description,News Type,Sentiment,Investor Reaction,GICS Sectors,Firm Age,Twitter Followers,Total Asset,ROA,Positive Consumer Perception,Negative Consumer Perception,clean_text,truncated_text
0,https://www.nytimes.com/2019/01/01/business/me...,2019.1.1,Netflix Blocks Show in Saudi Arabia Critical o...,Netflix has blocked an episode of its show _at...,['NFLX'],['Netflix Inc'],ESG,Negative,0.064605,Communication Services,23,21667744,33.976,6.07,-0.019300,0.011234,Netflix has blocked an episode of its show atr...,Netflix has blocked an episode of its show atr...
1,https://www.nytimes.com/2019/01/02/business/ju...,2019.1.2,Julie Sweet of Accenture Could See Her Future....,Why did you go to law school?\n \n I decided t...,['ACN'],['Accenture Plc Class A'],Non-ESG,Positive,-0.001291,Information Technology,31,551168,29.790,20.99,-0.014514,0.004481,Why did you go to law school I decided that I ...,Why did you go to law school I decided that I ...
2,https://www.nytimes.com/2019/01/02/business/te...,2019.1.2,"Tesla Reports Record Output, but Cuts Prices, ...",Tesla shares ended trading nearly 7 percent lo...,['TSLA'],['Tesla Inc'],Financial Performance,Negative,-0.010247,Consumer Discretionary,17,19694123,34.309,-1.94,0.001597,-0.014112,Tesla shares ended trading nearly percent lowe...,Tesla shares ended trading nearly percent lowe...
3,https://www.nytimes.com/2019/01/03/business/de...,2019.1.3,Bristol-Myers to Acquire Celgene in Deal Worth...,Bristol-Myers Squibb said on Thursday that it ...,['BMY'],['Bristol Myers Squibb'],Financial Performance,Neutral,-0.056534,Health Care,133,157472,129.940,3.83,-0.083046,-0.044012,Bristol Myers Squibb said on Thursday that it ...,Bristol Myers Squibb said on Thursday that it ...
4,https://www.nytimes.com/2019/01/03/business/de...,2019.1.3,Was Apple_ Warning a Surprise? Not to Its Supp...,"Apple_ stock fell 10 percent on Thursday, its ...",['AAPL'],['Apple Inc'],Financial Performance,Negative,-0.027306,Information Technology,44,9115548,338.516,19.42,-0.052604,0.017905,Apple stock fell percent on Thursday its worst...,Apple stock fell percent on Thursday its worst...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4470,https://www.nytimes.com/2022/10/06/us/politics...,2022.10.6,Biden Visits IBM to Promote Investments in U.S...,President Biden visited the Hudson Valley of N...,['IBM'],['International Business Machines Co'],Non-ESG,Neutral,-0.005044,Information Technology,112,713787,127.243,0.91,-0.093702,0.078402,President Biden visited the Hudson Valley of N...,President Biden visited the Hudson Valley of N...
4471,https://www.nytimes.com/2022/10/08/us/san-anto...,2022.10.8,Officer Fired After Shooting Teenager Who Was ...,A rookie San Antonio police officer was fired ...,['MCD'],['Mcdonalds Corp'],Non-ESG,Neutral,0.026175,Consumer Discretionary,68,177308,50.436,15.52,-0.011025,0.031923,A rookie San Antonio police officer was fired ...,A rookie San Antonio police officer was fired ...
4472,https://www.nytimes.com/2022/10/11/us/san-anto...,2022.10.11,Ex-Officer Is Charged After Shooting Teen Eati...,A former rookie San Antonio police officer was...,['MCD'],['Mcdonalds Corp'],Non-ESG,Neutral,0.013419,Consumer Discretionary,68,177308,50.436,15.52,0.008733,0.000723,A former rookie San Antonio police officer was...,A former rookie San Antonio police officer was...
4473,https://www.nytimes.com/2022/10/11/travel/luft...,2022.10.11,Lufthansa Says Passengers Can_ Use Apple AirTa...,Update: Lufthansa says Apple AirTags are once ...,['AAPL'],['Apple Inc'],Non-ESG,Neutral,-0.000911,Information Technology,47,9115548,352.755,33.76,-0.048449,0.046456,Update Lufthansa says Apple AirTags are once a...,Update Lufthansa says Apple AirTags are once a...


# ESG-BERT 돌리기

In [21]:
esg_bert = pipeline('text-classification', model = model, tokenizer = tokenizer)

def esg_bert_func(sent):
    try:
        return esg_bert(sent, top_k = None)
    except Exception as e:
        print(e)
        print(sent)
        pass

In [22]:
tqdm.pandas()
nytimes_df['ESG Bert Score'] = nytimes_df['truncated_text'].progress_apply(lambda x : esg_bert_func(x))

100%|███████████████████████████████████████| 4475/4475 [09:18<00:00,  8.01it/s]


In [23]:
nytimes_df['ESG Bert'] = nytimes_df['ESG Bert Score'].apply(lambda x : x[0]['label'])

In [24]:
nytimes_df.to_csv('/Users/parkjunhyeong/Desktop/Composed_Data_ver1.4.csv')

In [30]:
nytimes_df['GICS Sectors'].value_counts()

Communication Services    1530
Consumer Discretionary    1055
Industrials                513
Information Technology     476
Health Care                435
Financials                 219
Consumer Staples           115
Utilities                   66
Energy                      54
Real Estate                  8
Materials                    4
Name: GICS Sectors, dtype: int64